In [12]:
import sys
sys.path.insert(0,'/home/dan/Desktop/kaggle/kaggle-ieee-cis-fraud-detection/src/')
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from cross_val_labo import CrossValLabo
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'cross_val_labo'

In [57]:
train_set = pd.read_pickle('./train_set.pkl')
train_label = pd.read_pickle('./train_label.pkl')

In [14]:
bins = pd.cut(range(max([10, 10, 10, 10, 10, 10, 12, 13, 14])), 4, retbins=True, labels=False)
pd.cut([10, 10, 10, 10, 10, 10, 12, 13, 14], np.arange(0, 14, 4), labels=False)

array([ 2.,  2.,  2.,  2.,  2.,  2.,  2., nan, nan])

In [58]:
val_mask = train_set.TransactionDT >= 13000000
train_mask = train_set.TransactionDT < 13000000

X_val = train_set[val_mask].drop(columns=['TransactionDT'])
y_val = train_label[val_mask]
X_train = train_set[train_mask].drop(columns=['TransactionDT'])
y_train = train_label[train_mask]

del train_set
del train_label

In [59]:
X_train.first_group_nan.value_counts()

0    496424
1        12
Name: first_group_nan, dtype: int64

In [60]:
to_drop = [
    'M1', 'V14', 'V22', 'V27', 'V28', 'V41', 'V65', 'V68', 'V89',
    'V107', 'V119', 'V120', 'V121', 'V122', 'V241', 'V305', 'V325',
    'V327', 'V330', 'V11', 'V117', 'V240', 'V297', 'id_22', 'V6',
    'V21', 'V31', 'V113', 'V118', 'V173',  'V194', 'V328', 'V88',
    'V104', 'V106', 'V110', 'V116', 'V250', 'V337', 'id_26', 'V84',
    'V95', 'V109', 'V114', 'V141', 'V144', 'V191', 'V235', 'V269',
    'V339', 'V8', 'V9', 'V17', 'V32', 'V42', 'V48', 'V148', 'V153',
    'V252', 'V338', 'C3', 'V10', 'V18', 'V50', 'V105', 'V142',
    'V146', 'V174', 'V181', 'V193', 'V329', 'V1', 'V16', 'V98',
    'V101', 'V108', 'V111', 'V112', 'V319', 'V322', 'id_24', 'V7',
    'V15', 'V46', 'V97', 'V159', 'V183', 'V299',   'V334', 'M2',
    'V49', 'V69', 'V72', 'V125', 'V157', 'V166', 'V196', 'V197',
    'V226', 'V247', 'V255', 'id_04', 'id_10', 'V25', 'V30', 'V71',
    'V135', 'V151', 'V199', 'V227', 'V249', 'id_08']
len(to_drop)
X_val.drop(columns=to_drop, inplace=True)
X_train.drop(columns=to_drop, inplace=True)

In [67]:
lgb_params = {
                'objective':'regression',
                'boosting_type':'gbdt',
                'metric':'auc',
                'n_jobs':8,
                'learning_rate':0.05,
                'num_leaves': 2**7,
                'min_child_samples': 20,
                'max_depth':9,
                'tree_learner':'serial',
                'colsample_bytree': [0.5],
                'n_estimators':2**12,
                'max_bin':255,
                'seed': 1337,
                'silent': False,
            }
model = lgb.LGBMClassifier(**lgb_params)

In [62]:
model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_names=['train', 'val'],
    eval_metric='auc',
    verbose=10,
    early_stopping_rounds=50)

Training until validation scores don't improve for 50 rounds.
[10]	train's auc: 0.866906	val's auc: 0.848119
[20]	train's auc: 0.873894	val's auc: 0.855223
[30]	train's auc: 0.874384	val's auc: 0.857111
[40]	train's auc: 0.87643	val's auc: 0.858646
[50]	train's auc: 0.877511	val's auc: 0.859534
[60]	train's auc: 0.879072	val's auc: 0.860854
[70]	train's auc: 0.880051	val's auc: 0.861525
[80]	train's auc: 0.881014	val's auc: 0.862251
[90]	train's auc: 0.882086	val's auc: 0.863206
[100]	train's auc: 0.882584	val's auc: 0.863399
[110]	train's auc: 0.883952	val's auc: 0.864171
[120]	train's auc: 0.884681	val's auc: 0.864667
[130]	train's auc: 0.885478	val's auc: 0.865431
[140]	train's auc: 0.886369	val's auc: 0.866222
[150]	train's auc: 0.88757	val's auc: 0.866771
[160]	train's auc: 0.888897	val's auc: 0.867463
[170]	train's auc: 0.890056	val's auc: 0.867786
[180]	train's auc: 0.891607	val's auc: 0.868386
[190]	train's auc: 0.892717	val's auc: 0.868627
[200]	train's auc: 0.893459	val's auc

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=[0.7],
               importance_type='split', learning_rate=0.01, max_bin=255,
               max_depth=10, metric='auc', min_child_samples=30,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=4096,
               n_jobs=8, num_leaves=512, objective='regression',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, seed=1337,
               silent=False, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0, tree_learner='serial')

In [63]:
del X_train

In [64]:
preds = model.predict_proba(X_val)[:, 1]

In [65]:
score = roc_auc_score(y_val, preds)
score

0.8885181068448245

In [66]:
sorted(list(zip(X_val.columns, model.feature_importances_)), key=lambda x: x[1])

[('first_group_nan', 0),
 ('second_group_nan', 0),
 ('no_card_4', 0),
 ('no_card_6', 0),
 ('no_card_3', 4),
 ('fourth_group_nan', 11),
 ('id_12', 14),
 ('eigth_group_nan', 19),
 ('ninth_group_nan', 25),
 ('no_id_01_12', 29),
 ('no_dist1', 34),
 ('V302', 38),
 ('sixth_group_nan', 38),
 ('V182', 39),
 ('V2', 45),
 ('V91', 47),
 ('V90', 48),
 ('V100', 48),
 ('V4', 49),
 ('V237', 51),
 ('V59', 52),
 ('V185', 52),
 ('V286', 53),
 ('third_group_nan', 53),
 ('seventh_group_nan', 53),
 ('V138', 54),
 ('V29', 55),
 ('V236', 56),
 ('V92', 57),
 ('V103', 57),
 ('V93', 59),
 ('no_addr', 61),
 ('M8', 64),
 ('V70', 65),
 ('V60', 68),
 ('V137', 68),
 ('no_card_5', 68),
 ('M9', 70),
 ('V284', 70),
 ('id_21', 72),
 ('V254', 73),
 ('id_07', 74),
 ('V300', 78),
 ('V335', 79),
 ('V179', 83),
 ('V333', 84),
 ('V195', 85),
 ('V3', 86),
 ('V26', 86),
 ('V63', 86),
 ('V175', 86),
 ('V154', 89),
 ('V155', 90),
 ('V24', 92),
 ('V326', 92),
 ('V124', 94),
 ('V304', 94),
 ('addr2', 96),
 ('V123', 96),
 ('M7', 97)